In [1]:
import tensorflow as tf
import numpy as np
import os

#import models
import resnet_50_tl_all_wd_1e4
from utility import *

In [2]:
class Image_Classfier():
    def __init__(self):

        self.config = tf.ConfigProto(
            gpu_options=tf.GPUOptions(
                visible_device_list="0"
            )
        )
        
        self.train_layers = "fc"
        self.batch_size = 64
        self.resnet_size = 50
        self.model_name = "resnet" + "_" + str(self.resnet_size) + "_" + str(self.batch_size) + "_TL_All_RE_MX_WD_1e4"
        
        self.img_size = 128
        
        self.epoch_max = 18000
        self.epoch_save_model = 1000
        self.epoch_test = 100

        #dataset
        self.hand = hand_dataset()
    
    def build_model(self):
        tf.reset_default_graph()

        self.X = tf.placeholder("float", [None, self.img_size, self.img_size, 3])
        self.Y = tf.placeholder("float", [None, 2])
        self.x = tf.placeholder(tf.float32, shape=[None, 150, 200, 3])
        self.x_aug_train = imgs_augment(self.batch_size, self.x, test=False, re=True)
        self.x_aug_test = imgs_augment(self.batch_size, self.x, test=True)
        
        self.learning_rate = tf.placeholder("float", [])
        self.is_training = tf.placeholder("bool", [])

        # ResNet Models
        self.model = resnet_50_tl_all_wd_1e4.ResNetModel(self.is_training, self.resnet_size, num_classes=2)
        loss = self.model.loss(self.X, self.Y)
        self.train_op = self.model.optimize(self.learning_rate, self.train_layers)

        correct_prediction = tf.equal(tf.argmax(self.model.prob, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))


    def train(self):
        
        saver = tf.train.Saver()
        accuracies = {"train":[], "test":[]}
        
        with tf.Session(config=self.config) as sess:
            ckpt = tf.train.get_checkpoint_state("./model" + self.model_name + "/")
            epoch_pretrained = 0
            if ckpt:
                last_model = ckpt.model_checkpoint_path

                epoch_start_pos = last_model.find("/dcgan_model")
                epoch_end_pos = last_model.find(".ckpt")
                epoch_pretrained = last_model[epoch_start_pos + len("/dcgan_model"):epoch_end_pos]

                print(type(int(epoch_pretrained)))
                print("load: "+last_model)

                epoch_pretrained = int(epoch_pretrained)             
                saver.restore(sess, last_model)

            else:
                init = tf.global_variables_initializer()
                sess.run(init)

            num_iteration = 0
            num_iteration += epoch_pretrained
            lr = 0.001
            
            # load the pretrained weights
            self.model.load_original_weights(sess, skip_layers=self.train_layers)
            
            # train and test
            for j in range (self.epoch_max):
                if num_iteration > 16001:
                    break;

                if (num_iteration > 1000):
                    lr = 0.0001

                if num_iteration > 2000:
                    lr = 0.00001

                if num_iteration > 9000:
                    lr = 0.000001

                # training 
                num_iteration += 1
                Image_train, label_train = self.hand.extract_random_minibatch(self.batch_size, self.img_size,mixup=True)

                aug_Image_train = sess.run(self.x_aug_train, feed_dict={
                    self.x: Image_train,
                })
                aug_Image_train_np = np.array(aug_Image_train)
                aug_Image_train_np = aug_Image_train_np/255 - 0.5

                feed_dict={
                    self.X: aug_Image_train_np,
                    self.Y: label_train,
                    self.learning_rate: lr,
                    self.is_training: True,
                }
                _= sess.run([self.train_op], feed_dict=feed_dict)

                if num_iteration % self.epoch_test == 0:
                    # calculation of accuracy while training
                    acc_train_tmp = []      
                    for i in range(4):
                        Image_train, label_train, condition = self.hand.extract_next_minibatch(self.batch_size, self.img_size, i, test=False)

                        aug_Image_train = sess.run(self.x_aug_test, feed_dict={
                            self.x: Image_train,
                        })
                        aug_Image_train_np = np.array(aug_Image_train)
                        aug_Image_train_np = aug_Image_train_np/255 - 0.5

                        feed_dict={
                            self.X: aug_Image_train_np,
                            self.Y: label_train,
                            self.is_training: False,
                        }
                        acc = sess.run(self.accuracy, feed_dict=feed_dict)
                        acc_train_tmp.append(acc)

                    acc_train_ave = np.average(acc_train_tmp)
                    accuracies["train"].append(acc_train_ave)

                    acc_test_tmp = []
                    for i in range(4):

                        Image_test, label_test, condition = self.hand.extract_next_minibatch(self.batch_size, self.img_size, i, test=True)

                        aug_Image_test = sess.run(self.x_aug_test, feed_dict={
                            self.x: Image_test,
                        })
                        aug_Image_test_np = np.array(aug_Image_test)
                        aug_Image_test_np = aug_Image_test_np/255 - 0.5

                        feed_dict={
                            self.X: aug_Image_test_np,
                            self.Y: label_test,
                            self.is_training: False,
                        }
                        acc = sess.run(self.accuracy, feed_dict=feed_dict)
                        acc_test_tmp.append(acc)

                    #accuracy_summary = tf.summary.scalar("accuracy", accuracy)
                    acc_test_ave = np.average(acc_test_tmp)
                    accuracies["test"].append(acc_test_ave)
                    print(str(num_iteration) + "  train:" + str(acc_train_ave) + ", test:" + str(acc_test_ave))

                    path = "accuracy_" + self.model_name
                    if not os.path.isdir(path):
                        os.makedirs(path)                    
                    np.save(path + "/acc_" + str(self.resnet_size) + "_" + str(num_iteration) + "_train.npy", accuracies["train"])
                    np.save(path + "/acc_" + str(self.resnet_size) + "_" + str(num_iteration) + "_test.npy", accuracies["test"])

                    # save metrics
                    save_metrics(accuracies, self.model_name, num_iteration)
                    
                if num_iteration % self.epoch_save_model == 0:
                    path = "model" + self.model_name
                    if not os.path.isdir(path):
                        os.makedirs(path)
                    saver.save(sess, path + "/dcgan_model" + str(num_iteration) + ".ckpt")                

            sess.close()

# transition of accuracy during training

In [3]:
if __name__ == "__main__":
    resnet = Image_Classfier()
    resnet.build_model()
    resnet.train()

/home/ujitoko/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:6: DeprecationWarning: PyUnicode_AsEncodedObject() is deprecated; use PyUnicode_AsEncodedString() to encode from str to bytes or PyCodec_Encode() for generic encoding


AttributeError: 'hand_dataset' object has no attribute 'height'